<a href="https://colab.research.google.com/github/KhurramDevOps/Quarter-02/blob/master/Rag_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
%pip install -qU langchain-pinecone langchain-google-genai

In [62]:
# !pip install google-auth google-auth-oauthlib google-auth-httplib2
# !pip install langchain

In [63]:
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/my-gemini-api-444713-262839e78ed5.json"

In [136]:
from IPython.display import Markdown, display
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('pinecone_api')
pc = Pinecone(api_key=pinecone_api_key)

In [18]:
index_name = "practise-rag-project"

pc.create_index(
    name=index_name,
    dimension=768,
    metric='cosine',
    spec  = ServerlessSpec(cloud="aws",region="us-east-1"),
)

index = pc.Index(index_name)

In [119]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

GEMINI_KEY = userdata.get("GOOGLE_API_KEY_1")

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [120]:
vector = embeddings.embed_query("Building a Rag project! ")



In [121]:
vector[:5]

[0.005886509083211422,
 -0.01920737698674202,
 -0.01310189813375473,
 -0.03790365159511566,
 -0.003551947884261608]

In [122]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [123]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I have chocolate chip pancake and scrambled eggs for breakfast",
    metadata={"source": "personal", "meal": "breakfast"}
)

document_2 = Document(
    page_content="LangChain is a framework for building applications with LLMs (Large Language Models), such as GPT. It provides tools to manage chains, agents, and memory for building more advanced AI applications.",
    metadata={"topic": "LangChain", "category": "AI Framework"}
)

document_3 = Document(
    page_content="Agentic AI refers to autonomous AI systems that are capable of decision-making, learning, and adaptation in real-world environments without needing constant human intervention.",
    metadata={"topic": "Agentic AI", "category": "Artificial Intelligence", "importance": "High"}
)

document_4 = Document(
    page_content="In the latest world news, a new tech company has developed an innovative AI that is able to solve real-world problems faster than previous models, pushing the boundaries of automation and efficiency.",
    metadata={"topic": "Tech News", "date": "2025-01-02", "company": "Innovative AI Company"}
)

document_5 = Document(
    page_content="The use of AI in healthcare is growing rapidly. Recent advancements in diagnostic AI tools are helping doctors identify diseases more accurately and faster, significantly improving patient outcomes.",
    metadata={"topic": "Healthcare AI", "category": "Medical Technology", "impact": "Positive"}
)

document_6 = Document(
    page_content="LangChain offers a wide range of tools to work with LLMs. This includes support for document search, question-answering systems, and memory management to make intelligent agents more effective in real-world tasks.",
    metadata={"topic": "LangChain", "category": "AI Tools", "use_case": "Advanced workflows"}
)

document_7 = Document(
    page_content="Agentic AI is becoming increasingly important in industries like autonomous vehicles, robotics, and smart cities, where real-time decision-making and adaptability are crucial for success.",
    metadata={"topic": "Agentic AI", "category": "Industry Applications", "industries": ["Autonomous Vehicles", "Robotics", "Smart Cities"]}
)

document_8 = Document(
    page_content="A new world record has been set for the fastest internet speed, with researchers breaking through previous limitations using advanced fiber-optic technology, promising faster and more efficient global communication.",
    metadata={"topic": "Tech News", "category": "Innovation", "record": "Fastest Internet Speed", "date": "2025-01-02"}
)

document_9 = Document(
    page_content="In a recent study, AI-powered chatbots have been shown to outperform human customer service agents in resolving technical issues, reducing wait times and improving customer satisfaction.",
    metadata={"topic": "AI in Customer Service", "category": "Business Technology", "impact": "High"}
)

document_10 = Document(
    page_content="LangChain continues to evolve with new integrations, including support for databases, APIs, and external data sources, enabling more complex and efficient workflows for AI applications.",
    metadata={"topic": "LangChain", "category": "Development", "features": ["Database Integration", "API Support", "External Data Sources"]}
)

documents = [
    document_1, document_2, document_3, document_4, document_5,
    document_6, document_7, document_8, document_9, document_10
]


In [124]:
len(  documents)

10

In [125]:
from uuid import uuid4
uuid4()

UUID('313c8987-e230-4088-8530-7e71b0740cb8')

In [126]:
uuids = [str(uuid4()) for i in range (len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['c1c41343-a04a-4ef0-b206-62d1eb4d6a9a',
 '091fc645-3744-4ee8-99bb-967cb6802791',
 '487fbb31-94d9-4243-a882-99596a014a83',
 '97354cc2-00ed-4f5c-b8d4-147f19eee87d',
 '64e758fb-6da8-43f3-897c-ef319e6b08bf',
 '3ed98dca-6d78-498b-be7c-4cdeca7c0f30',
 '6ab198bd-91ae-4671-8536-a85136f7e602',
 '91dc49e9-fb06-45f7-8473-a5f93dee31b8',
 '0dda22cc-2730-461d-a98a-2b9d6b27ae6e',
 'eed046ec-76a3-4843-96b8-c10ab4098e71']

In [105]:
vector_result = vector_store.similarity_search(
    "Who won 2024 FIFA World Cup?", k=7
)
print(vector_result[0].page_content)

In a recent study, AI-powered chatbots have been shown to outperform human customer service agents in resolving technical issues, reducing wait times and improving customer satisfaction.


In [127]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import PromptTemplate

In [128]:
def user_answer(question):

  vector_result = vector_store.similarity_search(question, k=5)

  llm = ChatGoogleGenerativeAI(
      api_key=GEMINI_KEY,
      model = "gemini-2.0-flash-exp",
      max_tokens= 100,
      temperature=0.7
  )

  prompt1= PromptTemplate(
      input_variables=["question"],
      template = "Using this data {vector_result} . Answer the following question: \n\n{question}"
  )
  chain1= prompt1 | llm

  final_answer = chain1.invoke({"vector_result": vector_result, "question": question})

  return final_answer

In [129]:
response = user_answer("What's the latest news?")




In [139]:
display(Markdown(response.content))

Based on the provided data, here's the latest news:

*   **A new tech company (Innovative AI Company) has developed an innovative AI that can solve real-world problems faster than previous models.** This is highlighted in multiple documents with the same content and metadata.
*   **LangChain is evolving with new integrations, including support for databases, APIs, and external data sources.** This allows for more complex and efficient workflows for AI applications.

It's worth noting that the

In [142]:
response_1 = user_answer("What is Langchain and where it is used ?")


In [143]:
display(Markdown(response_1.content))

LangChain is a framework for building applications with Large Language Models (LLMs), such as GPT. It provides tools to manage chains, agents, and memory for building more advanced AI applications. LangChain also supports integrations with databases, APIs, and external data sources, enabling more complex and efficient workflows for AI applications.


In [148]:
response_2 = user_answer("What i like in breakfast , Tell me in Key points")

In [149]:
display(Markdown(response_2.content))

Okay, based on the provided data, here are the key points about what you like for breakfast:

*   **Chocolate chip pancakes:** You enjoy having chocolate chip pancakes.
*   **Scrambled eggs:** You also like to have scrambled eggs.

It appears you consistently have these two items for breakfast according to the provided documents.
